In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTqXalV0KWuOjMRtCNCbdKekTEmaSpwTKxal2pK8D26ogEttNxp&usqp=CAU',width=400,height=400)

the-scientist.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/hackathon/BCG_country_data.csv', encoding='ISO-8859-2')
df.head()

In [ ]:
na_percent = (df.isnull().sum()/len(df))[(df.isnull().sum()/len(df))>0].sort_values(ascending=False)

missing_data = pd.DataFrame({'Missing Percentage':na_percent*100})
missing_data

In [ ]:
categorical_cols = [cname for cname in df.columns if
                    df[cname].nunique() < 10 and 
                    df[cname].dtype == "object"]


# Select numerical columns
numerical_cols = [cname for cname in df.columns if 
                df[cname].dtype in ['int64', 'float64']]
print(numerical_cols)

In [ ]:
print(categorical_cols)

In [ ]:
for col in ('gni_per_capita_usd', 'year_of_gni_report', 'covid_19_test_total_per_thousand', 'covid_19_test_cumulative_total', 'hospital_bed_per_1000_people', 'female_population_percent', 'median_age', 'median_age_male', 'median_age_female', 'percentage_population_over_65', 'hearth_disease_disability', 'stroke_disease_disability', 'percentage_men_smokers', 'percentage_women_smokers', 'percentage_people_with_diabetes', 'population_per_km2', 'population_per_mi2', 'population'   ):df[col] = df[col].fillna(0)

In [ ]:
for col in ['income_group_type', 'covid_19_test_unit', 'covid_19_test_date_reported','region']:
    df[col] = df[col].fillna('None')

#Codes from  Thiago Panini https://www.kaggle.com/thiagopanini/the-best-model-for-a-bank-marketing-campaing

#Train and Test

In [ ]:
from sklearn.model_selection import train_test_split
# Splitting the data
X = df.drop('covid_19_test_cumulative_total', axis=1)
y = df['covid_19_test_cumulative_total'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_test: {X_test.shape}')

In [ ]:
# Returing features by dtype
numerical_features = [col for col, dtype in X_train.dtypes.items() if dtype != 'object']
categorical_features = [col for col, dtype in X_train.dtypes.items() if dtype == 'object']
print(f'Total of numerical features: {len(numerical_features)}')
print(f'Total of categorical features: {len(categorical_features)}')

# Splitting data by dtype
X_train_num = X_train[numerical_features]
X_train_cat = X_train[categorical_features]
print(f'\nShape of numerical training data: {X_train_num.shape}')
print(f'Shape of categorical training data: {X_train_cat.shape}')

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
# Class for splitting the data by dtype
class SplitDataDtype(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # Returing features by dtype
        self.numerical_features = [col for col, dtype in X.dtypes.items() if dtype != 'object']
        self.categorical_features = [col for col, dtype in X.dtypes.items() if dtype == 'object']
        
        # Indexing data
        X_num = X[self.numerical_features]
        X_cat = X[self.categorical_features]
        
        return X_num, X_cat

In [ ]:
# Creating object and calling the fit_transform method
dtype_splitter = SplitDataDtype()
X_train_num, X_train_cat = dtype_splitter.fit_transform(X_train)

print(f'Shape of numerical training data: {X_train_num.shape}')
print(f'Shape of categorical training data: {X_train_cat.shape}')

#Categorical Pipeline

In [ ]:
# Class por encoding the data
class DummiesEncoding(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # Collecting variables
        self.cat_features_ori = [col for col, dtype in X.dtypes.items() if dtype == 'object']
        
        # Applying encoding with get_dummies()
        X_cat_dum = pd.get_dummies(X)
        
        # Merging the datasets and eliminating old columns
        X_dum = X.join(X_cat_dum)
        X_dum = X_dum.drop(self.cat_features_ori, axis=1)
        self.features_after_encoding = list(X_dum.columns)
        
        return X_dum

In [ ]:
# Applying encoding on categorical data
encoder = DummiesEncoding()
X_train_encoded = encoder.fit_transform(X_train_cat)

print(f'Shape of X_train_encoded: {X_train_encoded.shape}')
X_train_encoded.head()

#Numerical Pipeline

In [ ]:
from sklearn.preprocessing import StandardScaler
# Scaling with StandardScaler() class
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_num)

# Looking at the first line
X_train_scaled[0]

In [ ]:
#With all the pipeline steps already defined, let's put everything in a complete Pipeline.
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Initial block code for splitting the data
dtype_spliter = SplitDataDtype()
X_num, X_cat = dtype_spliter.fit_transform(X_train)
numerical_features = dtype_spliter.numerical_features
categorical_features = dtype_spliter.categorical_features

# Numerical pipeline
num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Categorical pipeline
cat_pipeline = Pipeline([
    ('encoder', DummiesEncoding())
])

# Full pipeline
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, numerical_features),
    ('cat', cat_pipeline, categorical_features)
])

# Applying the complete pipeline on the training set
X_train_prep = full_pipeline.fit_transform(X_train)

# Returing features
categorical_features_encoded = full_pipeline.named_transformers_['cat']['encoder'].features_after_encoding
model_features = numerical_features + categorical_features_encoded

In [ ]:
# Result
print(f'Shape of X_train: {X_train.shape}')
print(f'Shape of X_train_prep: {X_train_prep.shape}')
print(f'Total features: {len(model_features)}')
print(f'\nFirst line of X_train_prep: \n\n{X_train_prep[0]}')

In [ ]:
# Applying the same pipeline for the test set
X_test_prep = full_pipeline.fit_transform(X_test)

print(f'Shape of X_test_prep: {X_test_prep.shape}')

In [ ]:
# Saving everything on a prepared set to feed some homemade classes
set_prep = {
    'X_train_prep': X_train_prep,
    'X_test_prep': X_test_prep,
    'y_train': y_train,
    'y_test': y_test
}

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, \
    GradientBoostingClassifier
import lightgbm as lgb

In [ ]:
cols_to_drop=['region']
df=df.drop(cols_to_drop,axis=1)
df.columns

In [ ]:
from sklearn.model_selection import train_test_split
# Hot-Encode Categorical features
df = pd.get_dummies(df) 

# Splitting dataset back into X and test data
X = df[:len(df)]
test = df[len(df):]

X.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
categorical_col = ('income_group_type', 'covid_19_test_unit', 'covid_19_test_date_reported')
        
        
for col in categorical_col:
    label = LabelEncoder() 
    label.fit(list(df[col].values)) 
    df[col] = label.transform(list(df[col].values))

print('Shape all_data: {}'.format(df.shape))

#Codes from Satya Muralidhar  https://www.kaggle.com/muralidhar123/heart-diesease-uci

In [ ]:
from sklearn.preprocessing import StandardScaler , LabelEncoder
scaler = StandardScaler()

In [ ]:
label = LabelEncoder()
train = df.iloc[:,:-1]
train

In [ ]:
train['covid_19_test_cumulative_total'] = label.fit_transform(train['covid_19_test_cumulative_total'])

In [ ]:
target = df['covid_19_test_cumulative_total']

In [ ]:
X_scaled = scaler.fit_transform(train)

In [ ]:
from sklearn.metrics import roc_curve , roc_auc_score , confusion_matrix , accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
grad = GradientBoostingClassifier()
X_train,X_test,y_train,y_test = train_test_split(X_scaled,target,test_size=0.4,random_state=120)
grad.fit(X_train,y_train)
y_pred=grad.predict(X_test)
accuracy_score(y_pred,y_test)

In [ ]:
auc = roc_auc_score(y_test,y_pred)
confusion = confusion_matrix(y_test,y_pred)
tp = confusion[0][0]
fp = confusion[0][1]
fn = confusion[1][0]
tn = confusion[1][1]

In [ ]:
from sklearn.metrics import plot_confusion_matrix
disp = plot_confusion_matrix(grad,X_test,y_test,cmap=plt.cm.Blues,normalize=None)
disp.confusion_matrix

In [ ]:
# finding accuracy 
accuracy = (tp+tn)/(tp+tn+fp+fn)
accuracy

In [ ]:
fpr , tpr , thresholds = roc_curve(y_test,y_pred)
plt.plot(fpr,tpr,color = 'darkblue',label = 'ROC')
plt.plot([0,1],[0,1],color='orange',linestyle='--',label="ROC Curve(area=%0.2f)"%auc)
plt.xlabel('False + ve rate')
plt.ylabel('True +ve rate')
plt.legend()
plt.show()

I have no idea how to fix the issues above. The primary intention was to make ROC and learning curves. Though I failed.

In [ ]:
#codes from Rodrigo Lima  @rodrigolima82
from IPython.display import Image
Image(url = 'data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxISERUSEhIVFhUWGBcYFxYXGRUcGBghHxgdGSAXIR8fISoiGB0lHhoYIjEhJSkrLi8uGCAzODMtNygtLi4BCgoKDg0OGxAQGjAlHSU1NS0tLS0uMi0tLS0vLS0tLS0tLS04LS0tLS0tLy0tLS0tLS0tLS0tLS0tLS0tLS0tLf/AABEIAJ8BPgMBIgACEQEDEQH/xAAcAAEAAQUBAQAAAAAAAAAAAAAABwMEBQYIAgH/xABEEAACAQMCAwQGBwcCBQQDAAABAgMABBESIQUTMQYiQVEHFBdSYZMyNVRxgdLTI0JzdJGzwjOhFRZygpIksbLRU2Ki/8QAGAEBAQEBAQAAAAAAAAAAAAAAAAECAwT/xAAjEQACAgEDBQEBAQAAAAAAAAAAAQITEQMSMiExUWGBQSIE/9oADAMBAAIRAxEAPwCcaUqEe2fpU4ha39xbxC30RPpXVG5b6IO5DjPXyrUYuTwiSkl3JupXPHtn4p7tr8p/1Ke2finu2vyn/UrdMjFsToelc8e2finu2vyn/Up7Z+Ke7a/Kf9SlMhbE6HpXPHtn4p7tr8p/1Ke2finu2vyn/UpTIWxOh6Vzx7Z+Ke7a/Kf9Sntn4p7tr8p/1KUyFsToelc8e2finu2vyn/Up7Z+Ke7a/Kf9SlMhbE6HpXPHtn4p7tr8p/1Ke2finu2vyn/UpTIWxOh6Vzx7Z+Ke7a/Kf9Sntn4p7tr8p/1KUyFsToelc8e2finu2vyn/Up7Z+Ke7a/Kf9SlMhbE6HpXPHtn4p7tr8p/1Ke2finu2vyn/UpTIWxOh6Vzx7Z+Ke7a/Kf9Sntn4p7tr8p/1KUyFsToelc8e2finu2vyn/Up7Z+Ke7a/Kf9SlMhbE6HpXPHtn4p7tr8p/1Ke2finu2vyn/UpTIWxOh6Vzx7Z+Ke7a/Kf9Sntn4p7tr8p/1KUyFsToelc8e2finu2vyn/Up7Z+Ke7a/Kf9SlMhbE6HpXPHtn4p7tr8p/1Ke2finu2vyn/UpTIWxOh6Vzx7Z+Ke7a/Kf9Sntn4p7tr8t/1KUyFsToelc8e2finu2vyn/Up7Z+Ke7a/Kf9SlMhbE6HpXPHtn4p7tr8t/1Ke2finu2vy3/UpTIWxOh6VFXoq9IV5xG8eC4EOhYGkHLRlORJGvix2w5/2qVaxKLi8M2mmsoVyz6TPra8/i/4LXU1cs+kz62vP4v+C100ORz1uJg7Hh085IghllK4LCKN3Iz0J0g48evlVK5t3jYpIjI46q6srD7wRkVuno9WY2XFhAJDLyrbRytXMzzW+jp3zjPStrj4SLheHRcRRpbyO2vJTC7HmShWBhik/eORkgde42fGuznhnJQyiG6VLPDOzNrcycPkubRbWWVbtpLVVkRZOV9A8snUuepAPeA8qtpLHh4u7ZmtJDmK45wSzukhDKyiObkMdTouohwpO+npmliFRGCgk4AyTsAKvL3hFzCA01tPEpOAZIpEBPllgATWb7RcNa24qImWFSJYCBAGWPB0EEKxJTOclSdiT4YqSO3kOu34upF1Goe3kZ594HCsDot+gRicZPeycDywc+3sLTzkhxeEXJi54t5zDjPNEUnLx568acfjXq14LdSpzIrWeSPfvpFKybde8ARtW/8ApPi4ibqRrb1j1H1ZeWYS/q/J5Pezjue9sd8Y+FbDdcKluOJRRpJfRWTQw+pS2TBbdFWIs3MOCASwwPE5HhipZ0yVaabIQr7W0dg7KG4vHtZ1VjcRTRxO3WOXGpJAfA90j8a2uPs9FHIYVtrZms7W39aeWKeZjNMdRAiiOZTjAz+6K05pGVp5Iroalzi3Ze3tpeIy29ktzJE1qI7ZlkdI1lUM78tTqbfVj3cHwrC8Y4SIeHI8XDFkadb1riQrKxtDG7KI1YHEXKAOdX0tB86liLWaPxHh8tvIYp42jkGCVYYIyMj/AGNWtS52os7W2bitw1pFMbf1DlLKXKjWgU53yRuCVzvgV5k7LxR3940dtbmBBaHRJFPPyzKuSkcUZydR/ebZcDHjhYWsicKcE4OB1PgP/qq99w+WEqJY2QuiyKGGCVbo33HB/pUs8Rs0toONWttZxyLFNakIVkc6ZFDEkBskR5Zl8t85xVtPwKHLSrbi4lh4ZZSRW7a3V2csrSFQdThQB3Rtv91SwVETUqWJeCWcbTTSWUYYcLju3tTrCxS69wBnVGCAMr1AyPGtR7f2cKtaTQwpCLmzhneNM6FZtWdIPQbDb4fGtKeXgzLTwsmq0qYeG9mrd0tUbhycibhqzTXvfzHIIs5DZ0oc4J8W1jwU1jIeF2TNwi0a1iX1yGCSafLCQnc6BvhdZGknqdQ6YFSxFqIyAzsKMCDgjBHhUrw2USG3uZeHxWksfE47eNNMiLLGTjUVZu+yHvCQbZX7xVd+DxS3vFZ57WNJIGXlIYJ5VdXmcG5MSsGmLAY1KdI3ONqlhaiIK9FTjODg9D4VJ3/BrQNxG4tLI3DxeqiK0min7nM+m/KOHZfFc9B8Ko9pJGPBIgLGOMLc3KOFWUm2Ik89XdY/RJbr8M1d5K+hGtKUrocxXpBuPvFea9R9R94oDIX92yyyKFiADuAOTBsAxAH0Kpw3MjsFVYiWIAHJg8f+zb76p8T/ANeX+I//AMjVbgd6sMyuyqRuDqUsBnbJHiPMeKlh41proXP9YyZ1OCuNQkkhVwAdIt4OrDUucqGGVBbBUHAOATscHOWMcodUDI6DaOJSD3wRlVGen+1SLx/iMc5iJQbd4s2oaGBzy8phVOzd9iF7yNkgVoHErkSCZlC41RKCowDgOMgeA8APBQo8K8+hOc4KU1h+PB6/9ENOGFB5KV9cFHKqsQAC4/ZQn90eJXJ/GqAvX92P5MH5K+8V/wBVv+3/AOAq3ikKsGHVSCPwOa9CSwePLybXDwY4AZo9WcNpjtQFOgvjdCWOFJ8NgT5ZxV/AyCRWEZBiV0YRQqd5oxnKDY4JGxOx+Nbjwvj5SFmgdCHBYK0bMVYYBDNkBe6mcHA+ODtqHELlGVo48lYoFTJIOTzogdxsRsBkbHcjYiuaydnjBt/oA+s5f5WT+9DXQNc/egD6zl/lZP70NdA15tbkdNLiK5Z9Jn1tefxf8FrqauWfSZ9bXn8X/BauhyJrcTA2l7LESYpZIyepjdkJ+/SRmvPrMmvmcx+ZnPM1Nrz56s5z8c1luy3AlvHdGkmUqFwIbaSdmySCSFICKuBuTvnatit/RoxnmhkuSOVcRW4aKB5STIgkEjKGHKj0kZYk75HhXdyin1OKjJo0me+leQSvLI0gxiRncuMbjDE5GPDeqzcZui4lNzOZFGFkMsusA+AbVkD4ZrbOC+j9LqWSGK6lLLLJEHFnKYgU8XkD6U1EHABbwPjivdn2QN1Dw+MvBCZIr5y6wnXmKQAiRg37XyDADSM7NTfEu2Ro0szMxdmZmJyWYksT5kncn41Wu+IzygCWeWQA5AkkdgD54YnBrbLfsEksloIL1ZIbtJ2SUwsjZhOGQRs+WYnp3h0PlvW4Z2UsuVxHnzzK1sIdLPbyI8WojLNHr7xJJTGSMDUDvTfEmyRpi8RmEfJE0vK//HzH5f8A450/7V7tuK3EaGOO4mRD1RJJFQ/9oOKuOznC1uZuUzyL3Sw5UDzuxBA0hFI8ydRIAx8RWyXvo+5Es/PutFvBDDM0xhbXiViiJytWQ5ZWBBbbH9K5RXRhKT6mlRSMrBlYqynIZSQQR4gjcGq8fEp1dpFnlWRtmcSOHbPgWBy34mtgPZBCtrJFdCSO6u/VUblMuBqVebhmz4/Q26dav73sraRcOnladjcxXstsCI3w5RW0w414XXgPzN9OrTvjNTcgoSNRi4pcK5kWeZXIClxJIHIAwF1A5IwBt8BXlL+YI8YmlCOSXQO4VyepZc4Y/fW48f8ARtLa280xlZmgEbSKYXSMh8A8uUtiUrkZ2FYns92aintZrue7FvFA6I37JpCdYOMBWBznAx5EnwpujjI2yzgwc19K4YPLIwfTrDO516fo6snvafDPTwr2nFLgMzi4mDMArMJJAzADAUnOWAG2DtW3t6NZhaGcyPrFt61o5D8rTjVy+dqxzdO+nTjwz41b33YXlGzU3DFrt7dA4gJgAlIGVl14kZM7qQpPh0pviNsjV04lOGdhPKGcaXYSPqcYxpY5ywxtg0j4jMrrIs0odVCq4kcMqjoobOQo8htW4f8AI6IyvDdx3BhvYba4RoHVEZpQnQt+2TV3SBjO+D418412ShVbq6mu44FS7nt0ijt30s6rqVVAc8tDv1zpA6nNN8S7ZGnG9lJdjLJmQYkOtsuPJjnvjYbHPSqcs7tp1OzaVCrqJOlR0UZ+io8htW8t6P0RI52uJjDz4opOZaSwkhzgPGGbLqThfAjOfhTjXYu29dvViuTFbWihpmMTu0RZtKwqC+ZifeyKb4jZI1zi/aWeeOGLU0ccVvFblEdwkgjzhmXOCd/KsVJcO2nU7NoAVMsx0gbhVz9EDyFbnH6P1LsTeqLcWi3i3HJbvRlsEGPVlWAycZOdh47eLXsEJjbvBd67acXJacxFeSIc5LIXzuACNx1+FFKKDjJmpXl/NKQZZpJCuymR3cr8BqJx0HTyqr/xa55nO9Yn5oGkSc2TmAe7qzqx8M1ZZ8unx60rWDGWXcfE51kMqzzCRtmkEkgc/AsDk9B1PgK8C9l0NHzZNDnU6a20ufeZc4Y9NzvtVvSrgmWKUpQCvUfUfeK816Q7j7xQGTu+HSPLIygaTNIqlnjXUwbJVdRGogEZx01DzGfK8CuCyqEUs6hlHNgyVKGQN9PYFAW3x0q/urjUDG6QyIk00kR9YhX/AFCMq3e7yHShx3WG+++Ku4uOuGgbloeQiogN3DgAQmHIGdmIwSfNRjG9NzOmyJgrrhdwqs7oxWIxox1K2nWuuMbE91l+iRsdhnNerqykhWeKVdLq0QZcqSDhjjYkZwRWaPHpu8QkGWUKzNPAxbSiqrtlu9Krokms9WXpisdxu8MpmlYIpkaLurLHJ9FCpPdOfAHfxNTL/Q4pHmazWSSQmTRp07aHb9wY+j0zgivDcJQAnnHYZP7KTbBYHPljT/X7iavEvmU9wRMudakzQqctDyiGBbcYP0SNjnwJBrf8amDF1WEOTEctPA4/ZlyMhn3JD4Y5BJBbOpmJZZdsTHNZ8tZdMz4wARynGvIOB/8Ar1IyfePnvawxsqzBlKnljYgg/wCtF51mF4nIIXgCRaGXTk3EJcDMrfS1Zzqmb4YGCNyTQ4petKspYRqCHICyxOxaS5SVhs2SBvgY2APiSaZZGkbb6APrOX+Vk/vQ10DXP3oA+s5f5WT+9DXQNeXW5HbS4iuWfSZ9bXn8X/Ba6mrln0mfW15/F/wWrocia3Etez/aV7WKeHkxSxz6CySax3kJZTlGUkZO6k4P9c7xwbt9BKk73UkcMs1wkkiGK4eMosCRYQxMGVzp/fJX4VoPC+FLLb3Ux5mqAQ6QiAodble+c9zpt5nNVbzshxCJNclnMq6xHkgY1E6QvXxYgA9MnGa7SjF9zlFyRnoO3y2zqlraxGGCeWS15vN1xq5OoEB8MSCcE5Kg+NY+y7czxCDEcR5CXSLnXuLhtTE7+B6Vfdp/R/LbyQW8EdxNPJGXkJWMRbBdWg5yApbBL7ZK771r0HZm9e4a1W2lM6DLR4wVHvEnYDcb5wcjFEoNBueS4tu0uIbWCS3ilithMFDmQFua4ctqVgVZSNiKyJ7fzNNcSSwQSpcRpE8LB9AWP6G+rUSDuSSc58NsW1x2EvY7Q3ckTKqyMjoR31C9ZSOmnVleucjy3rze9kJ2vZ7Wzhnm5OnOpVRgGUN3snSpJJwM7gZp/DH9ln2a4+9m0hEaSJNE0UiOXGpSQdmUhlOR1B8/gRmH9IEzSMZLa3eJ4IreS3IflssRZkI72pWBY43P9d6w9n2WvpXeOO1lZ42COunBRiCQDnpkAnPTFLTsrfSyyQpayGSHAlTABTPQEkgb+G+/hmq1FkTkjLcP7eyRLoFpaMqz+sQqUbTA2ADoGrptkZyQSTvti0ftc7Q3ULwxOtzM9wCdeqGRgQXQg+AO2fKsNY8OmmlEEUbPKSQEA72RnP3Ywc56Yq/m7KXyy8hrWXm8sy6AASUGxcYJDDO22aYihuky54/2q9cVmltLf1h9Ae5XmB204wdOrQrEKATjcZ6VYW3GnSzmswqlJnjkZjnUCnQDfGDVWfsvepOls1rKJ3GpI8DLDfcYOMDByc7Y3q57TdmHsoLR5RIk04uOZG4A0cuUIuMeDKQ2cnrttRbew/rufb7tVzoFjmtbeSVIRAlwebzFQDA2DaWcZ2Yj8DWQ4Z20Cm1h5MVvAl1bzzmPmtqMbqSwUs2gYBJVBuf96Ft2ShW3gnvb5bU3ILQx8mSUlQcCRirDQpyMddjnzA+WnZSFbeGe9vltRcajCohklJUHBkbBXQuSMddjnzFT+TS3l5xzt6zysLeCCOP1z1ksiOrXBSTVGzgnbICkgYJOTt0rF8T7XTTIUKRrm8a9BGdnIxp3OCoq77QdhZbSC4leVWa3mSN0VdtEiho5w2roxIXTjYg7nFe5uwxWORmuAJIrJbx4+X9HV0hJ1/SxuWx4jaotiD3s+cY7fTTpKoghiMskUzunMLF4zkN32IA2HdxgYPnSbt7I1xNM1rbFbmMJcw4k0TEHIkPeyrjwIO3xO9Xfs+jLwwDiCetTwpNHA8Miq2pSwTmhiAdmG43x0rTYbQmUQsVjbXoYyEKqHOkljvgA5yd+lVKL7EbmjP3PbadzN+ziVJbX1RY1DBIowcgLv167nPX8KueB9qFt+EXlpzP2s8gCJpbuoygSvq+iAyjTpznO/jXy57DML+1soblZfWokmWXQyKFZXcnSSS2FQnwzkDAqhf8AZu1ijt7oXjzWcryI0iQ6JVZATpCO5B1Y2JIwOo8389i/33NXpWw9rOAxWy200EjvDdRGRBKoWVcEAhgDg9QQRsf9zr1bTz1ObWHgUpSqQUpSgFKUoBSlKAUpSgFKUoBSlKAkv0AfWcv8rJ/ehroGufvQB9Zy/wArJ/ehroGvJrcj1aXEVyz6TPra8/i/4LXU1cs+kz62vP4v+C1dDkTW4nns3xiGGzv4ZCQ84teWApIPLmLtkjZcA+PWs72i7WW87cZ0yOy3a2otwVfflkE7EdzG5GcVpHDOHy3EqQQIXlc4RAVGTgnqSANgTknwrzxCxkgleGZCkkZ0upxkHyyMg/eNq7uKyclNpElf862TXd0WYcq4treJZJIXkRWjG6vHszKTkbeQqzm7UW05vLee60x3EFtFHcw20iInJdn5fK1Fyp1ac+IHhUc5oDUrQsZuc3EbNuHS2vrUuuK7NxCzxOeevKWPB3/ZHIJ7x6AdSdszx3tNYXpv4GuHgjnuIZ4pxDI+oJCkZRkGGGCpIztneo/4Tw2S5dki05VHkOplUYUZO7EDPwr3wThEt3IYodOoI8h1HA0qMn8fhRwRVN+Dc+0/bSC4h4ksTSK1w1osOQQzrEArMxGy5xnBPQislJ2tsZb25klnU2s4t+ZDLbSyc7lxBSykEGKRSCASMdDvgVFea+ahStEsZsHZm4slvtVyrC2PN0g6m05B0awp1Oo2DAHf7q3WTtpZIYuXLjl2N3b5ihkjUSOylAq7lVOkkHO22cHaorzV3c8PeOGGZtOibmaMMpPcbS2QN13O2etWUE31JGbRvnBe19oiWUckkg02V1ayyKrFoTIwKuPF8Bf3c4zWC7YcStntrC2t55JvVVnRnkRkJ1OrLgEnC4BAGdgADitVzTNFBJ5Dm2sG8XHEeHX1rZrc3EttNaRCBgsLSiVF+iVIOEbHveJPkCUt9w6+tLSO6uZbaS1QwkCFpRLHkEaSp7j489s56jFaOTWV4BwGW8MgiMarEmuSSVwkaLnGSx6ZNHFL9Kpt/hucPbW0uL+89aDJY3UKw40lnAjxy2IXO/0+mcFh5Zqw/wCbYpJuLyyEqbuF44BpY5wQqKcfR7ir12rR22r5mmxEsZJr9ouGes2l+08zyW1vDGLZYSNTxqdzIx0hct4e749K1C6496ziKaK1hV5dclwkGZl1OWYlgcyAajttnAFYHNM0UEg5tkgcd4xZtc2M9rxGRHt4YrcyerSZQRo452Ce8GJClN9mO56V47W9rY5ltLdJzMIZudLccgRKSWGAkQwcKuonOCxNaEDVaytZJnWOJGkds6UQFmOAScAddgT+FNiLvZlO2XFfWr2aYTNMrMdDsGXu9QoViSigk4H/AN1haUzWksdDm+ryKVe8J4ZJcycqLTq0s3eZVGFGTuxAz8K9cC4TJeTCGHTrKsw1HAwqlj/sKZRcMsKV8DD+tfc1SClM0BoBSlKAUpSgFKUoBSlKAkv0AfWcv8rJ/ehroGufvQB9Zy/ysn96Guga8mtyPVpcRXLPpM+trz+L/gtdTVyz6TPra8/i/wCC1dDkTW4mR9HUUUUV7ezTGFUi9XjkCl2WSbbWqggllUf/ANfCvXpSSOVra/hk5iXMOl30lS0kOI2Yqd1JGnY+7WjYpiu+3rk47umCXTIZLrh9lF6tEvqUFwW5ELyPJyXXA17M+Pog7A5JzWXPDo3u4mMMTTPw261idLcBnSRAplCfswwyQfIZG2KgrSPIVf8AB+KyWpkaHSDLE8L5APcfGofA7DesvT8GlqL9JYktlM0Au44Bef8ADr03KqsWARjlEhe6H06unx8KuuzqlIrfkJbiybhzZk/YiU3BjYyDOdZfY6h0wD5VBwUeQpoHkKV+xb6Nv9FYT1uQyIjqtpcNpkAKEhQcEH8f963PsVdLcWpuuRz7l7g+tRwx2eWQKAqMsuAkJUdUxuSc9ah4ivjKD1ArUobiR1MEt9morSaI3DxRxrwu5unaI6CzQsryRoSMhysgVRuR3djvXjsvFbSrwl7hItci8TcDEYDS8/KAhu6cZOkNtkConKjyppHl1qV+y2eiYbjlNxC3t7i1aJrm2uLeSSdbUO5bBjl0wnSrKV0hsAnUMZxVfiltZwwSXEaxFuHW8tkQQp5kzJCI5MfvYZ5cnrtULhR5CmkeVSv2LPRvno9t2NpdtZpG/EFMIiDiJmEee+yCTu58z5Y+FbVc8TS3PFxbCAFLe3d1RI2j5xIWQDIIZemV3AbPjmoZZQeor7pHl0qvTy8kWphYN09GNuGa75SxteLb5tFk0HLau8VD90uBjGfM+Ga2O24PNcwJzI7cXEHE45LsKYEVI+ShJODpK+YXOW1bdaigjPWvmkeQquGXkKeESl2gvha2fEXhjg5h4pPCC0aMVRoQSFz9Hpj8TWUnuLduLwWckduIFhSVAEiGucwYUsx2Y42AO2VTxqGQo8qaR0xUrLYTQscZ4hAJ7MrILS85huVtQZ8AFCyQnT3RqGcAkEeVUey/Fw8/BbpxAktwt7DMyxxoCEYiMYA7pyqgEYPeI8cVDoQeQppHl161KxablweDPHI04msa/tcTIRGI88vuAhe5pJ5eR0Od/Gty4neRw3dibqzdS8skMkk62Y1xyDTjTCcFUZlYMRsAcE5NQ4BXwKB0AquGSKeCYrPhsFreW3DcRyci1u5ZXIQ6nkDFQTjcqijH/VV92ORkjsTAlt6mbJjJIeVzfWSj6wDnXr8MDbGvPQYg/SOmBQoPIVHp5/S2ExcDt7deF25jtWuIWtnNyEFpp5uDqaR5CJEdDjTg9AMU4HNGTw20MNuYrjhzGcmNC7FUcjL9Rggn7yfhiHSo64FNI8qV+xb6JNNs3/DLQ26QGyaEG/kIhLiTX3ySTzA6j6Gn/wBsVkPSZFElpMqWrcoPD6rOBZiGNcjIjKHmOrLnIIO++1RFpHXFAo64FWvr3Fi8H2lKV0OQpSlAKUpQClKUBJfoA+s5f5WT+9DXQNc/egD6zl/lZP70NdA15NbkerS4iuWfSZ9bXn8X/Ba6mqI+1fohmu7ye5W7jQSvqCmNiRsBjOrfpTSkovqNSLa6EH0qW/YXP9ui+U/56ewuf7dF8p/zV3th5ONUiJKVLfsLn+3RfKf81PYXP9ui+U/5qWw8iqRElKlv2Fz/AG6L5T/mp7C5/t0Xyn/NS2HkVSIkpUt+wuf7dF8p/wA1PYXP9ui+U/5qWw8iqRElKlv2Fz/bovlP+ensLn+3RfKf81LYeRVIiSlS37C5/t0Xyn/NT2Fz/bovlP8AmpbDyKpESUqW/YXP9ui+U/56ewuf7dF8p/zUth5FUiJKVLfsLn+3RfKf81PYXP8AbovlP+elsPIqkRJSpb9hc/26L5T/AJ6ewuf7dF8p/wA1LYeRVIiSlS37C5/t0Xyn/NT2Fz/bovlP+elsPIqkRJSpb9hc/wBui+U/561vjnoq4nbZKxCdB+9CdR/8Dhs/AA1VqRf6R6ckaRSqk8LIxR1ZGHVWBVh94O4qnWzApSlAKUpQClCcda2HgfYniF3gw2smk/vuOWn35bGof9OajaXcqTfY16lStbeg66Kgvdwo3iqo7gfjlc/0qr7C5/t0Xyn/AD1i2Hk3VIiSlS37C5/t0Xyn/NT2Fz/bovlP+alsPIqkRJSpb9hc/wBui+U/56ewuf7dF8p/z0th5FUjH+gD6zl/lZP70NdA1HHo49GsnDLp7h7lJQ0LRaVRlIy6NnJY+5j8akevPqtOWUd4JqOGKUpXM2KUpQClKUApSlAKUpQClKUApSlAKUpQClKUApSlAKUpQClKUBY8V4Nb3K6LiCOVfAOqtj4jPQ/EVoHHPQtYy5a2kkt28s8yP+jd7+jCpNpWlJrsRxT7nOPHPRHxK3yY0S4TzibDfijYP4KWrRry3eJ9EqNG46o6lW/ocGux68PCpIJUEruCQCR8R5V0Wu/05vRX4cu8D7AcSu8GO1dVP78v7Nfv73eYf9INSDwP0HDZry6J80gGB92twSR/2ipkpUetJlWlFGvcC7EcPs8GC1jDDpIw1yf+TZI/DFbDSlc28nQUpSoBSlKAUpSgFKUoBSlWHH+b6rNyQTLy30AEAltJxjIIznpkGgL+vmaj2y4TdI9ubhWdVe51ZHOAAMKR7KseCVjYqxBwN/HAuMcTZkLxhjDKUTWkOHKw3OLkEDKCQvAu2nGGGACa3s9mN/o3ulR5CnEVLunOBleLXK8cPMwLc7hApGkS4U4XOPh3qv7qfiqmRowXOZVSMrGI9rMOj5+lvcApu2MMR5Gmz2N/o3SvlaiJb/nwqjy8nEZLyxR6nPNbmK4VRoAj06SNPUk6ulWdtYTC1iV4ZjMskRvSCczAOS5VtX7RScNpH7mVxvppt9l3G90qPp7C9jDTW6Sd6WeKKMtvFFLGgWQqTsEnTWF6qkhG3QVBwl45yHhmkgWWQKBqYlfVLdEI3276vvtg5O3Wmz2Td6N9pWO7OxTJawLOSZVjQOSdRzjfJ/ePmfE1kawzSFKUoUUpSgFKUoBSlUL67WKN5XzpRSxwMk48APEnoB4k0BXpWLg7QWzCP9qqmVVdVbZsNnGR4HIIx5jFIu0Voyh1uIipOAQwx9EN/TSynPTDA+NBkylKwsPam0ZtIlwf2xJZXUDkvofJIAGG/r4VVvuPwxxpLkusjlF0YO4V2YHJAGBG+c+Ix1q4YyZWsT2pv5ILcyRfT5kCAYBzrmSMgAkDVhjjJAzjNW0XayBiNIcgso1HlqO80KhsMwbBM6eGdiMZwD8uu2FoizOZAViXVkFcOdDvpTfvNpjY74GMHON6YJk1+X0hmGNeZGkknKnkYByjAxrcPoKlWAP/AKco2lm0sT4YzkW7TzCUJyk16uXo5vc1GYIG18vV0IOMfD41nX47bLnVMg0lQQSNiWCgfE6iq4HiQOprynaG0ZOYLiMoMZYMMbjIOfLAO/TY+VX4T6YjiPadha2l0mhFnCMyEq0mGUEKgLLzSCRkDvEfRBO1Y3ifbWTDCNVTQ0gbDBm2huW5bAriKTMCtp72x8uu0ntDaDV/6iPutpbvdG1Mun79SOMeakVdy38SxiUyLyyAQ+cggjIIx1BG+1T4U1k9sJFcKYEKltOrmnVgTxW7Np5eM5lUgZ8DuNqtR6QMpI6QIwj5rEiVhlY4lkJGY8h+8RpYDBG5Hhsv/MFplh6xFlRkjUNh3P6/6kfzF8xXhO0ERhjmAciVzHGoALOwLDA3x0RmySBgZOKvwn089ouMiKyuLiJ4y0aShCSCpkXUgj67tzBo0jfO3WsT/wA5MZGiSBCwl5QBlIKnn8nMgCHlavpoO9qXfas0OMWsQ0mRIsAEocKRqYDGPPU6jbO7DxNfF7QQ8qOViUWSVogWwMMrOuTvgA8tsH4ioUwPaDtZIlnaXMRjjNwpbEhUqCbd5VXJZQe8qjzIzjeql320MYDNEgDSiNcyMrtshY6SmFYa9kLZOD5GszD2ktWQOZkUFGk7xA7oDEsfhpR2+5SfA1WTjdsdOJ4++Sq94bnUFx/5Mo+9lHiKvwfTXX7d4zmANg3IKxya3JhjEgRQF/aMy9SDpUjBbV3a923bUtycwoBMzKjrKGj7hDSHUFwAsRZ98ZMbDbGayh7V2mtUEykNq74I0AqEOCc+KyKwxkY3zjFfeJcSsCf28kR0akIcjbUjahg9QUST8FbyNPhPpmUcEAg5BGQR4/GvVfFIIyOlfayaFKUoBSlKAUpSgFKUoBSlKAUpSgFKUoBSlKAUpSgFWnEuHpOnLkyV1IxAOM6WDAH4ZAP4Vd0oDBRdlLdc6dYBKnGrIGmZpwBnw1u23ltVGTsXaMFDKToCgatDbCNItOGUjdY03xnI2IzWx0q5ZMIwM/ZS3cEMZCrc/K6hjEziV16dOYoceR26bVXTs5biOKLQCkMhlVdKBSxDgkqqhT/qMdgN8GsvSmWMGDuuy0EknMJfJfWQGwCdcTgHboGhjOPgfA4q3bsXbaOXqmA0csYfcJoaPlg4+jocjz2BznetkpTLGEYKLsnbLI0iqQzOsmQEzkSLL9LTqILqCQSR5Yqjddi7WRBG3M06UTGobhFdR1GM4dt+oOMYrY6UyxhGFi7NQKwYa8q7OO94tJJKfD3pX/DHlVS47PwvapaHVyo1RV3GSEAAByCG2HiPj1xWWpTIwYD/AJRtsAftBpwVOrdSOThht1Bt4j+B65q6PAYuVHEC45TmSNw3fViWJbJGDnW4wQQQxFZWlMjBrUvZ6zW4DySnnPpYB5F1Ny3jkz01MA0Sk+A3xgGrqG0tOTEyyqYklMqPrUqXd3GNXQ5aQgDzwK98V4Q808MmtAkWpgjIxJcqyCTUHH0VdwFxjvk9cYxknY8vbervcsFMssjFFGo61YBcyFycFi2WLNkDBGBigo3fZ7hgMgkljBSIxyani1IrqyBiSNUR0ysAQQDkdavn4BZSyrLqV5NXMXJjfOdDggEHAGlGBXB+ODVvP2NEhZ5Zi0rSRPrUMoUK1u0iqA2VLm3XvZyuRjpvV4Z2YkgaJlmQ8vSpzGe8ojWM9HGGOnOdxknamfZDGcG7O20MDRz3cbhAYWYGNNOtI42Vjndzylxq3AwN8Crx+A8PV5ZWnAIc80mVBoZkljwx8DpuJANW/wBHyFer3spJK8kjTRa3clW5J2TlSwiMjmb6VmYgjHeLE7NgfJeycjO0nPj16hyyYmOlFjliWJhzO/gTO2oYJck9MKGfYwbTDGFUKOgAA/AYr3VGztxHGkYJIRVUE9TgY3+O1VqyaP/Z',width=400,height=400)

cebm.net

Das War's, Kaggle Notebook Runner: Marília Prata    @mpwolke